In [ ]:
import astropy.coordinates as coord
from astropy.io import fits
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
%matplotlib inline
import numpy as np

from gala.mpl_style import hesperia, laguna, turbo_r
from pyia import GaiaData

from totoro.data import APOGEEDataset, datasets
from totoro.config import plot_config as pc
from totoro.config import fig_path, galcen_frame

In [ ]:
apogee = APOGEEDataset('../../data/apogee-parent.fits')

In [ ]:
data_name = 'apogee-rgb-loalpha'
d_lowa = datasets[data_name]

d_all = apogee.filter({'LOGG': (d_lowa.t['LOGG'].min(), d_lowa.t['LOGG'].max()),
                       'TEFF': (d_lowa.t['TEFF'].min(), d_lowa.t['TEFF'].max()),
                       'FE_H': (-3, 1)},
                      low_alpha=None)

In [ ]:
len(d_lowa)

## Typical uncertainties:

In [ ]:
error_g = d_lowa.g.get_error_samples(size=32)
vtan = error_g.vtan.reshape(error_g.vtan.shape[0:1] + (2, -1))

In [ ]:
dist_errs = np.std(error_g.distance, axis=1)
np.median(dist_errs)

In [ ]:
np.median(np.std(vtan, axis=0), axis=-1)

## Plots:

In [ ]:
vstep = 1.
zstep = 50 / 1e3
vzz_bins = (np.arange(-pc['vlim'], pc['vlim']+1e-3, vstep),
            np.arange(-pc['zlim'], pc['zlim']+1e-3, zstep))

In [ ]:
norm = mpl.colors.LogNorm(vmin=1, vmax=3e2)
cmap = 'laguna_r'

fig = plt.figure(figsize=(12, 10),
                 constrained_layout=True)

gs = GridSpec(nrows=2, ncols=2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1])

bins = (np.arange(-2., 0.75, 0.01),
        np.arange(-0.18, 0.4, 0.005))

H, xe, ye = np.histogram2d(d_all.get_elem_ratio('M_H'), 
                           d_all.get_elem_ratio('ALPHA_M'), 
                           bins=bins)
H_sub, xe, ye = np.histogram2d(d_lowa.get_elem_ratio('M_H'), 
                               d_lowa.get_elem_ratio('ALPHA_M'), 
                               bins=bins)

ax1.pcolormesh(xe, ye, H.T, norm=norm, cmap='Greys', rasterized=True)
cs = ax1.pcolormesh(xe, ye, H_sub.T, norm=norm, cmap=cmap, rasterized=True)

ax1.plot(apogee._mh_alpham_nodes[:, 0], 
         apogee._mh_alpham_nodes[:, 1],
         marker='', color='#777777', ls='--', lw=1.5)
ax1.set_xlim(-1.5, 0.7)
ax1.set_ylim(-0.12, 0.32)
ax1.set_xlabel('metallicity, [M/H]')
ax1.set_ylabel(r'alpha abundance, [$\alpha$/M]')

ax1.text(-0.2, -0.097, f'{len(d_lowa)} stars', 
         fontsize=14, ha='left', va='bottom', color='#777777')

# ---

galcen = d_lowa.c.transform_to(galcen_frame)
x = galcen.x.to_value(u.kpc)
y = galcen.y.to_value(u.kpc)
z = galcen.z.to_value(u.kpc)
vz = galcen.v_z.to_value(u.km / u.s)

xy_size = 2.1
bins = (np.arange(-8.122 - xy_size, -8.122 + xy_size + 1e-3, zstep),
        np.arange(-xy_size, xy_size + 1e-3, zstep))
H, xe, ye = np.histogram2d(x, y, bins=bins)
ax2.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

ax2.set_xlabel('Galactocentric $x$ [kpc]')
ax2.set_ylabel('Galactocentric $y$ [kpc]')

H, xe, ye = np.histogram2d(vz, z, bins=vzz_bins)
ax3.pcolormesh(xe, ye, H.T, norm=norm, cmap=cmap, rasterized=True)

ax3.set_xlabel(f'Galactocentric $v_z$ [{u.km/u.s:latex_inline}]')
ax3.set_ylabel('Galactocentric $z$ [kpc]')

cb = fig.colorbar(cs, ax=ax1, aspect=30)
cb.set_label('$N$ stars')

ax1.set_title('APOGEE–Gaia Parent Sample', fontsize=24, pad=12)

# fig.tight_layout()
fig.set_facecolor('w')

fig.savefig(fig_path / f'{data_name}-mh-am-xy.pdf', dpi=300)